# Structural Bioinformatics WS1516
## Global Sequence Alignment Implementation
**Author**: Charlie Hoyt

**Date**: 18 Jan 2016

This notebook outlines the implemention of the [Needleman-Wunsch](https://en.wikipedia.org/wiki/Needleman%E2%80%93Wunsch_algorithm) algorithm to perform a global alignment on sequence fragments from hemoglobin and myoglobin. It makes use of the [BLOSUM40](ftp://ftp.ncbi.nih.gov/blast/matrices/BLOSUM40) scoring matrix from the NCBI for sequence alignments.


### BLOSUM40 notes from the NCBI:
- Matrix made by matblas from blosum40.iij
- column uses minimum score
- BLOSUM Clustered Scoring Matrix in 1/4 Bit Units
- Blocks Database = /data/blocks_5.0/blocks.dat
- Cluster Percentage: >= 40
- Entropy =   0.2851, Expected =  -0.2090

In [1]:
blosum40_data = """A  R  N  D  C  Q  E  G  H  I  L  K  M  F  P  S  T  W  Y  V  B  Z  X  *
A  5 -2 -1 -1 -2  0 -1  1 -2 -1 -2 -1 -1 -3 -2  1  0 -3 -2  0 -1 -1  0 -6 
R -2  9  0 -1 -3  2 -1 -3  0 -3 -2  3 -1 -2 -3 -1 -2 -2 -1 -2 -1  0 -1 -6 
N -1  0  8  2 -2  1 -1  0  1 -2 -3  0 -2 -3 -2  1  0 -4 -2 -3  4  0 -1 -6 
D -1 -1  2  9 -2 -1  2 -2  0 -4 -3  0 -3 -4 -2  0 -1 -5 -3 -3  6  1 -1 -6 
C -2 -3 -2 -2 16 -4 -2 -3 -4 -4 -2 -3 -3 -2 -5 -1 -1 -6 -4 -2 -2 -3 -2 -6 
Q  0  2  1 -1 -4  8  2 -2  0 -3 -2  1 -1 -4 -2  1 -1 -1 -1 -3  0  4 -1 -6 
E -1 -1 -1  2 -2  2  7 -3  0 -4 -2  1 -2 -3  0  0 -1 -2 -2 -3  1  5 -1 -6 
G  1 -3  0 -2 -3 -2 -3  8 -2 -4 -4 -2 -2 -3 -1  0 -2 -2 -3 -4 -1 -2 -1 -6 
H -2  0  1  0 -4  0  0 -2 13 -3 -2 -1  1 -2 -2 -1 -2 -5  2 -4  0  0 -1 -6 
I -1 -3 -2 -4 -4 -3 -4 -4 -3  6  2 -3  1  1 -2 -2 -1 -3  0  4 -3 -4 -1 -6 
L -2 -2 -3 -3 -2 -2 -2 -4 -2  2  6 -2  3  2 -4 -3 -1 -1  0  2 -3 -2 -1 -6 
K -1  3  0  0 -3  1  1 -2 -1 -3 -2  6 -1 -3 -1  0  0 -2 -1 -2  0  1 -1 -6 
M -1 -1 -2 -3 -3 -1 -2 -2  1  1  3 -1  7  0 -2 -2 -1 -2  1  1 -3 -2  0 -6 
F -3 -2 -3 -4 -2 -4 -3 -3 -2  1  2 -3  0  9 -4 -2 -1  1  4  0 -3 -4 -1 -6 
P -2 -3 -2 -2 -5 -2  0 -1 -2 -2 -4 -1 -2 -4 11 -1  0 -4 -3 -3 -2 -1 -2 -6 
S  1 -1  1  0 -1  1  0  0 -1 -2 -3  0 -2 -2 -1  5  2 -5 -2 -1  0  0  0 -6 
T  0 -2  0 -1 -1 -1 -1 -2 -2 -1 -1  0 -1 -1  0  2  6 -4 -1  1  0 -1  0 -6 
W -3 -2 -4 -5 -6 -1 -2 -2 -5 -3 -1 -2 -2  1 -4 -5 -4 19  3 -3 -4 -2 -2 -6 
Y -2 -1 -2 -3 -4 -1 -2 -3  2  0  0 -1  1  4 -3 -2 -1  3  9 -1 -3 -2 -1 -6 
V  0 -2 -3 -3 -2 -3 -3 -4 -4  4  2 -2  1  0 -3 -1  1 -3 -1  5 -3 -3 -1 -6 
B -1 -1  4  6 -2  0  1 -1  0 -3 -3  0 -3 -3 -2  0  0 -4 -3 -3  5  2 -1 -6 
Z -1  0  0  1 -3  4  5 -2  0 -4 -2  1 -2 -4 -1  0 -1 -2 -2 -3  2  5 -1 -6 
X  0 -1 -1 -1 -2 -1 -1 -1 -1 -1 -1 -1  0 -1 -2  0  0 -2 -1 -1 -1 -1 -1 -6 
* -6 -6 -6 -6 -6 -6 -6 -6 -6 -6 -6 -6 -6 -6 -6 -6 -6 -6 -6 -6 -6 -6 -6  1"""
blosum40_data = iter(blosum40_data.strip().split("\n"))
header = next(blosum40_data).split()
blosum40 = {}
for line in blosum40_data:
    row_index, *row_data = line.split()
    for column, row_datum in zip(header, row_data):
        blosum40[row_index, column] = int(row_datum)

In [5]:
import numpy as np

def align(a, b, s, gap=-2):    
    """
    scores
    At position scores[i,j], this matrix stores the maximum score for
    choosing the best of matching, inserting, or deleting. This 
    allows for memoizing of smaller cases and alows this algorithm
    to make use of the dynamic programming paradigm
    """
    scores = np.zeros((len(a) + 1, len(b) + 1), dtype=int)
    
    """
    breadcrumbs
    At each step, the algorithm chooses the maximum of matching, 
    inserting, or deleting. The breadcrumbts stores for element (i,j) 
    the tuple representing the location of the maximum, at either:
    (i-1,j-1), (i-1,j), or (i,j-1)
    
    """
    breadcrumbs = {}
    
    # initialize
    for i in range(len(a) + 1):
        scores[i, 0] = gap * i
        breadcrumbs[i, 0] = (i - 1, 0)
    for j in range(len(b) + 1):
        scores[0, j] = gap * j
        breadcrumbs[0, j] = (0, j - 1)
    
    # calculate optimal substructures
    for i, a_char in enumerate(a, start=1):
        for j, b_char in enumerate(b, start=1):
            # calculate the positions and scores to examine for (i,j)
            recurrences = {
                # matching
                (i - 1, j - 1) : scores[i - 1, j - 1] + s[a_char, b_char],
                # insertion
                (i - 1, j)     : scores[i - 1, j] + gap,
                # deletion
                (i, j - 1)     : scores[i, j - 1] + gap
            }
            # store the position of the highest score for later
            breadcrumbs[i, j] = max(recurrences, key=recurrences.get)
            # store the highest score for later
            scores[i, j] = recurrences[breadcrumbs[i, j]]
    
    
    print("A:", a, len(a))
    print("B:", b, len(b))
    print(scores)
    
    """
    Now that the results have been built, they need to be assembled.
    This makes use of the breadcrumbs by first looking at the final
    position (where the score is necessarily the highest), and backtracking.
    Because the algorithm is backtracking, it builds the aligned strings
    backwards and they are ultimately reversed before printing.
    
    Through each iteration, the current position (a_current, b_current) is
    compared to the previous position stored in the breadcrumbs at
    breadcrumbs[a_current, b_current]. The type of match is calculated, and
    the appropriate character is concatenated to the alignment strings,
    sa, and sb. 
    """
    # initialize
    a_aligned, b_aligned = "", ""
    a_current, b_current = len(a), len(b)
    
    while 0 != a_current or 0 != b_current:
        a_predecessor, b_predecessor = breadcrumbs[a_current, b_current]
        if a_current - 1 == a_predecessor and b_current - 1 == b_predecessor:
            # matching
            print("M ", end="")
            a_aligned += a[a_predecessor]
            b_aligned += b[b_predecessor]
            
        elif a_current - 1 == a_predecessor and b_current == b_predecessor:
            # insertion
            print("I ", end="")
            a_aligned += a[a_predecessor]
            b_aligned += "-"
        elif a_current == a_predecessor and b_current - 1 == b_predecessor:
            # deletion
            print("D ", end="")
            a_aligned += "-"
            b_aligned += b[b_predecessor]
            
        print("({},{}) => ({},{})".format(a_current, b_current, a_predecessor, b_predecessor))
        a_current, b_current = a_predecessor, b_predecessor
    
    a_aligned = a_aligned[::-1]
    b_aligned = b_aligned[::-1]
    print("{}\n{}".format(a_aligned, b_aligned)) 

In [6]:
s = {
    ('A','A'): 10,
    ('A','G'): -1,
    ('A','C'): -3,
    ('A','T'): -4,
    ('G','A'): -1,
    ('G','G'): 7,
    ('G','C'): -5,
    ('G','T'): -3,
    ('C','A'): -3,
    ('C','G'): -5,
    ('C','C'): 9,
    ('C','T'): 0,
    ('T','A'): -4,
    ('T','G'): -3,
    ('T','C'): 0,
    ('T','T'): 8
}
align("AGACTAGTTAC", "CGAGACGT", s, gap=-5)

A: AGACTAGTTAC 11
B: CGAGACGT 8
[[  0  -5 -10 -15 -20 -25 -30 -35 -40]
 [ -5  -3  -6   0  -5 -10 -15 -20 -25]
 [-10  -8   4  -1   7   2  -3  -8 -13]
 [-15 -13  -1  14   9  17  12   7   2]
 [-20  -6  -6   9   9  12  26  21  16]
 [-25 -11  -9   4   6   7  21  23  29]
 [-30 -16 -12   1   3  16  16  20  24]
 [-35 -21  -9  -4   8  11  11  23  19]
 [-40 -26 -14  -9   3   6  11  18  31]
 [-45 -31 -19 -14  -2   1   6  13  26]
 [-50 -36 -24  -9  -7   8   3   8  21]
 [-55 -41 -29 -14 -12   3  17  12  16]]
I (11,8) => (10,8)
I (10,8) => (9,8)
I (9,8) => (8,8)
M (8,8) => (7,7)
M (7,7) => (6,6)
I (6,6) => (5,6)
I (5,6) => (4,6)
M (4,6) => (3,5)
M (3,5) => (2,4)
M (2,4) => (1,3)
M (1,3) => (0,2)
D (0,2) => (0,1)
D (0,1) => (0,0)
--AGACTAGTTAC
CGAGAC--GT---


In [7]:
hemoglobin = "KTEAEMKASEDLKKHGT"
myoglobin = "HGSAQVKGHG"
align(hemoglobin, myoglobin, blosum40, gap=-8)

A: KTEAEMKASEDLKKHGT 17
B: HGSAQVKGHG 10
[[   0   -8  -16  -24  -32  -40  -48  -56  -64  -72  -80]
 [  -8   -1   -9  -16  -24  -31  -39  -42  -50  -58  -66]
 [ -16   -9   -3   -7  -15  -23  -30  -38  -44  -52  -60]
 [ -24  -16  -11   -3   -8  -13  -21  -29  -37  -44  -52]
 [ -32  -24  -15  -10    2   -6  -13  -21  -28  -36  -43]
 [ -40  -32  -23  -15   -6    4   -4  -12  -20  -28  -36]
 [ -48  -39  -31  -23  -14   -4    5   -3  -11  -19  -27]
 [ -56  -47  -39  -31  -22  -12   -3   11    3   -5  -13]
 [ -64  -55  -46  -38  -26  -20  -11    3   12    4   -4]
 [ -72  -63  -54  -41  -34  -25  -19   -5    4   11    4]
 [ -80  -71  -62  -49  -42  -32  -27  -13   -4    4    8]
 [ -88  -79  -70  -57  -50  -40  -35  -21  -12   -4    2]
 [ -96  -87  -78  -65  -58  -48  -38  -29  -20  -12   -6]
 [-104  -95  -86  -73  -66  -56  -46  -32  -28  -20  -14]
 [-112 -103  -94  -81  -74  -64  -54  -40  -34  -28  -22]
 [-120  -99 -102  -89  -82  -72  -62  -48  -42  -21  -29]
 [-128 -107  -91  -97  -88  -80